In [ ]:
# TIGER 미국배당다우존스 458730
# TIGER 미국채10년선물 305080
# KODEX 골드선물(H) 132030
# TIGER 미국나스닥 100133690
# KODEX 2차전지산업 305720

In [ ]:
!pip install finance-datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
df_krx = fdr.StockListing('ETF/KR')
df_krx.head()

,Symbol,Category,Name,Price,RiseFall,Change,ChangeRate,NAV,EarningRate,Volume,Amount,MarCap
0,459580,6,KODEX CD금리액티브(합성),1054605,2,80,0.01,1054607.0,0.8675,136048,143477,88610
1,357870,6,TIGER CD금리투자KIS(합성),55275,2,5,0.01,55273.0,0.8576,50114,2769,69989
2,069500,1,KODEX 200,33040,2,95,0.29,33113.0,-9.5912,2115940,69698,55111
3,360750,4,TIGER 미국S&P500,20520,2,95,0.47,20496.0,11.2109,1411499,28961,53988
4,423160,6,KODEX KOFR금리액티브(합성),108700,2,10,0.01,108693.0,0.8630,13053,1418,45100


In [ ]:
stock_list = ['TIGER 미국배당다우존스', 'TIGER 미국채10년선물', 'KODEX 골드선물(H)', 'TIGER 미국나스닥100', 'KODEX 2차전지산업']
select_df = df_krx[df_krx['Name'].isin(stock_list)].set_index('Symbol')
select_df

,Category,Name,Price,RiseFall,Change,ChangeRate,NAV,EarningRate,Volume,Amount,MarCap
Symbol,,,,,,,,,,,
133690,4,TIGER 미국나스닥100,128275,2,875,0.69,128142.0,10.2266,105849,13581,39355
458730,4,TIGER 미국배당다우존스,12765,5,-30,-0.23,12717.0,10.8367,578575,7386,16371
305720,2,KODEX 2차전지산업,14020,2,245,1.78,14040.0,-11.4718,903406,12538,9842
132030,5,KODEX 골드선물(H),16100,2,110,0.69,16094.0,3.6964,44893,721,1739
305080,6,TIGER 미국채10년선물,12190,2,30,0.25,12185.0,1.2489,29023,353,1536


In [ ]:
import pandas as pd

data = pd.DataFrame()
for code_name, values in select_df.iterrows():
  df = fdr.DataReader(code_name, '2014')[['Close']].rename(columns = {'Close' : values['Name']})
  data = pd.concat([data, df], axis = 1)

In [ ]:
data = data.dropna()

In [ ]:
Change_df = data.pct_change().dropna()
Change_df.head()

,TIGER 미국나스닥100,TIGER 미국배당다우존스,KODEX 2차전지산업,KODEX 골드선물(H),TIGER 미국채10년선물
Date,,,,,
2023-06-21,0.008653,-0.002002,-0.007534,-0.009109,0.011989
2023-06-22,-0.013466,0.000528,0.007074,-0.002798,0.002194
2023-06-23,0.018203,0.003588,-0.003598,-0.007615,0.004378
2023-06-26,-0.003294,-0.005152,-0.015475,0.004039,0.003923
2023-06-27,-0.015096,0.002114,-0.015543,0.001207,-0.004776


In [ ]:
cum_change_df = (Change_df + 1).cumprod() - 1
cum_change = cum_change_df.iloc[-1]
cum_change

,2024-11-20
TIGER 미국나스닥100,0.501417
TIGER 미국배당다우존스,0.345485
KODEX 2차전지산업,-0.519692
KODEX 골드선물(H),0.275248
TIGER 미국채10년선물,0.082593


In [ ]:
# # CAGR = (목표금액 / 투자금액) ** (1/투자년도) - 1
investment_period = len(cum_change_df) / 252
CAGR = (1+cum_change) ** (1/investment_period) - 1
CAGR

,2024-11-20
TIGER 미국나스닥100,0.343320
TIGER 미국배당다우존스,0.240495
KODEX 2차전지산업,-0.412900
KODEX 골드선물(H),0.193123
TIGER 미국채10년선물,0.059326


In [ ]:
daily_std = Change_df.std()
annual_std = daily_std * np.sqrt(252)
annual_std

,0
TIGER 미국나스닥100,0.176869
TIGER 미국배당다우존스,0.119274
KODEX 2차전지산업,0.452722
KODEX 골드선물(H),0.141286
TIGER 미국채10년선물,0.075962


In [ ]:
port_weights_list = []
port_return_list = []
port_std_list = []

for i in range(20000):
  weights = np.random.random(len(stock_list))
  port_weights = weights / np.sum(weights)
  port_return = (port_weights * CAGR).sum()
  covariance_matrix = np.cov(Change_df.T, ddof=0) * 252  # 연간 공분산
  port_std = np.sqrt(port_weights.T @ covariance_matrix @ port_weights)
  port_weights_list.append(port_weights)
  port_return_list.append(port_return)
  port_std_list.append(port_std)

In [ ]:
cor_name = Change_df.columns.to_list()

port = {}
port['return'] = port_return_list
port['risk'] = port_std_list

for i in range(len(cor_name)):
  port[cor_name[i]] = [weight[i] for weight in port_weights_list]

In [ ]:
port_df = pd.DataFrame(port)
port_df.head()

,return,risk,TIGER 미국나스닥100,TIGER 미국배당다우존스,KODEX 2차전지산업,KODEX 골드선물(H),TIGER 미국채10년선물
0,0.112340,0.105200,0.259821,0.298229,0.159421,0.003577,0.278952
1,0.100533,0.067344,0.081893,0.225226,0.093249,0.158302,0.441331
2,-0.019645,0.196486,0.161498,0.248563,0.402866,0.152293,0.034779
3,0.055660,0.129515,0.179861,0.280157,0.247677,0.085641,0.206664
4,0.004071,0.160740,0.140435,0.055844,0.319263,0.340135,0.144322


In [ ]:
import plotly.express as px
fig = px.scatter(port_df, x="risk", y="return",
                 hover_data=port_df.columns[2:])
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
port_df['sharp'] = port_df['return'] / port_df['risk']

In [ ]:
port_df.sort_values('sharp', ascending = False)

,return,risk,TIGER 미국나스닥100,TIGER 미국배당다우존스,KODEX 2차전지산업,KODEX 골드선물(H),TIGER 미국채10년선물,sharp
12565,0.208360,0.075812,0.251306,0.237722,0.000700,0.261043,0.249228,2.748366
16737,0.193921,0.070617,0.166060,0.270151,0.000831,0.290621,0.272337,2.746074
12859,0.208669,0.076025,0.226913,0.245692,0.001253,0.306293,0.219849,2.744759
10005,0.209443,0.076337,0.236601,0.288221,0.001985,0.236507,0.236686,2.743666
3411,0.215870,0.078830,0.177731,0.397681,0.000886,0.257405,0.166298,2.738418
...,...,...,...,...,...,...,...,...
16345,-0.204182,0.259115,0.007755,0.006024,0.583878,0.066675,0.335668,-0.787997
15571,-0.296487,0.366040,0.009180,0.053681,0.806608,0.095335,0.035195,-0.809987
17879,-0.316077,0.382371,0.071691,0.012426,0.843781,0.003299,0.068804,-0.826625
6916,-0.326855,0.392920,0.030694,0.055414,0.866298,0.031030,0.016564,-0.831861
